## Steps for building waypoint graph for CARLA


### First Iteration
1. Generate original graph (version 0) from build_map_graph.py
2. Use graph_search.py to evaluate every points' path to goal point. It will store the points that cannot found the path to goal point using original version of graph
3. The points that cannot found the path to goal will be store in {Town}_not_found.txt


### Second Iteration
1. Run unit_test_usage.ipynb, and input "town name" and "each case in the {Town}_not_found.txt"
    - If path found:
        - Path will automatically add to the graph
    - If path not found:
        - (Option 1) Run manual_build.py to manually add new edge to the graph
        - (Option 2) If the distance between start and end is far, use manual_build.py to find the middle point, then use manual_add.py to find the path from "start to middle" and "middle to end"
2. Next
    - (Option 1) Keep Iterating through each testcase until solving every test case in {Town}_not_found.txt
    - (Option 2) Rerun graph_search.py to evaluate every points' path to goal point. Generate new {Town}_not_found.txt

In [2]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


In [95]:
##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)


if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)


##################### Input line in the {town}_not_found.txt #####################

start_end_string = input("Enter each list with start-end point in the not found list")

# Example input:
# [(547, 645), (3871, 1346), 'Forward', '45_45_196']


entries = re.findall(r'\([^)]+\)', start_end_string)
start = entries[0]
start = start.strip('() ')
x0, y0 =  map(int, start.split(','))
first_tuple = tuple([x0, y0])

end = entries[1]
end = end.strip('() ')
x, y =  map(int, end.split(','))
second_tuple = tuple([x, y])

print(latest_file)
print(index)
print(first_tuple)
print(second_tuple)

all_whites_pos = iter_all_white_points(waypoint_map)
start = first_tuple
end = second_tuple


new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(0, 0, 255), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(255, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "", True)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    add_path(G, final_path, new_file_name)
    print(new_file_name, "Store!")
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", new_img)




graph_list/Town03/3_graph_v39.adjlist
39
(726, 1445)
(682, 1997)
Start point: (726, 1445)
End point: (682, 1997)
Finding Path...
Get!!!
found!!!
[(726, 1445), (724, 1448), (721, 1451), (719, 1454), (717, 1457), (715, 1460), (713, 1463), (711, 1466), (709, 1470), (707, 1473), (705, 1476), (703, 1479), (701, 1483), (698, 1489), (696, 1493), (693, 1499), (692, 1503), (691, 1506), (689, 1510), (688, 1513), (687, 1517), (685, 1520), (684, 1524), (683, 1528), (682, 1531), (681, 1535), (680, 1538), (679, 1542), (678, 1546), (678, 1549), (677, 1553), (676, 1557), (676, 1560), (675, 1564), (674, 1568), (674, 1572), (674, 1575), (673, 1579), (673, 1583), (673, 1586), (673, 1590), (672, 1594), (672, 1598), (672, 1601), (672, 1605), (672, 1609), (672, 1613), (673, 1617), (673, 1621), (673, 1625), (673, 1629), (673, 1633), (673, 1637), (673, 1641), (673, 1645), (673, 1649), (673, 1653), (674, 1657), (674, 1661), (674, 1665), (674, 1669), (674, 1673), (674, 1677), (674, 1681), (674, 1685), (674, 168

In [94]:
############### Quick test for other start point with the same goal point

start, end = (4596, 3900), (4630, 2197)

fp = nx.shortest_path(G,  start,  end)
print(fp)



[(4596, 3900), (4597, 3896), (4598, 3892), (4601, 3881), (4602, 3878), (4604, 3866), (4605, 3863), (4607, 3852), (4608, 3848), (4610, 3837), (4611, 3833), (4613, 3822), (4613, 3818), (4615, 3807), (4616, 3803), (4617, 3792), (4618, 3788), (4619, 3776), (4620, 3773), (4621, 3761), (4621, 3758), (4622, 3746), (4623, 3742), (4624, 3731), (4624, 3727), (4625, 3716), (4625, 3712), (4626, 3701), (4626, 3697), (4626, 3686), (4626, 3682), (4626, 3670), (4627, 3667), (4627, 3655), (4627, 3651), (4627, 3639), (4627, 3635), (4627, 3623), (4627, 3619), (4627, 3607), (4627, 3603), (4627, 3591), (4627, 3587), (4627, 3575), (4627, 3571), (4627, 3559), (4627, 3555), (4627, 3543), (4627, 3539), (4627, 3527), (4627, 3523), (4627, 3511), (4627, 3507), (4627, 3495), (4627, 3491), (4627, 3479), (4627, 3475), (4627, 3463), (4627, 3459), (4627, 3447), (4627, 3443), (4627, 3431), (4627, 3427), (4627, 3415), (4627, 3411), (4627, 3399), (4627, 3395), (4627, 3383), (4627, 3379), (4627, 3367), (4627, 3363), (4627